In [1]:
import numpy as np
import pandas as pd
from datasets import load_dataset

In [2]:
lyric_data = load_dataset("cs182project/182-final-project")
lyric_data

DatasetDict({
    train: Dataset({
        features: ['id', 'conversations'],
        num_rows: 12933
    })
    test: Dataset({
        features: ['id', 'conversations'],
        num_rows: 681
    })
})

In [4]:
lyric = pd.DataFrame(lyric_data['test'])
lyric

,id,conversations
0,0,"[{'from': 'human', 'value': 'Write me a song c..."
1,1,"[{'from': 'human', 'value': 'Write me a song c..."
2,2,"[{'from': 'human', 'value': 'Write me a song c..."
3,3,"[{'from': 'human', 'value': 'Write me a song c..."
4,4,"[{'from': 'human', 'value': 'Write me a song c..."
...,...,...
676,676,"[{'from': 'human', 'value': 'Write me a song c..."
677,677,"[{'from': 'human', 'value': 'Write me a song c..."
678,678,"[{'from': 'human', 'value': 'Write me a song c..."
679,679,"[{'from': 'human', 'value': 'Write me a song c..."


In [8]:
def extract_prompt(s):
    return s[0]['value']

def extract_target(s):
    return s[1]['value']

In [9]:
lyric['prompt'] = lyric['conversations'].map(extract_prompt)
lyric['target_response'] = lyric['conversations'].map(extract_target)
lyric

,id,conversations,prompt,target_response
0,0,"[{'from': 'human', 'value': 'Write me a song c...",Write me a song called HETHEN with the followi...,[Intro: Offset]\nSo icy\nYeah\n[Chorus: Offset...
1,1,"[{'from': 'human', 'value': 'Write me a song c...",Write me a song called Why Would I Stop? with ...,"[Intro]\nYeah, woah\nThis what they been waiti..."
2,2,"[{'from': 'human', 'value': 'Write me a song c...",Write me a song called Still Got That Rock wit...,[Verse]\nNigga I be hard on a ho 'cause a bitc...
3,3,"[{'from': 'human', 'value': 'Write me a song c...",Write me a song called Little Things with the ...,"[Verse 1: Zayn, \nZayn & Liam\n, \nLiam\n]\nYo..."
4,4,"[{'from': 'human', 'value': 'Write me a song c...",Write me a song called Light Up (Cover) with t...,[Verse 1: Lil' Pope]\nOnly videos connect me ...
...,...,...,...,...
676,676,"[{'from': 'human', 'value': 'Write me a song c...",Write me a song called Trumpets with the follo...,[Chorus]\nEvery time that you get undressed\nI...
677,677,"[{'from': 'human', 'value': 'Write me a song c...",Write me a song called Love on the Brain (Cove...,"[Verse: Jesy, \nAll\n & \nJade\n]\nAnd you got..."
678,678,"[{'from': 'human', 'value': 'Write me a song c...",Write me a song called In Her Lane with the fo...,"[Hook: Ty Dolla $ign]\nGirl, keep shitting on ..."
679,679,"[{'from': 'human', 'value': 'Write me a song c...",Write me a song called Fight Night with the fo...,"[Intro: Takeoff]\nIf you know me, know this ai..."


In [11]:
generated = pd.read_json("fastchat/llm_judge/data/lyric/model_answer/lyre-lm.jsonl", lines=True)
generated

,question_id,answer_id,model_id,choices,tstamp
0,0,KxXAZ6LaGeLrTBAPTZxgu2,lyre-lm,"[{'index': 0, 'turns': ['[Intro: Offset & 21 ...",1.700809e+09
1,1,LMAq8FLz2p4etTLHiJ9m8i,lyre-lm,"[{'index': 0, 'turns': ['[Intro] ( I made it )...",1.700809e+09
2,2,BP8pScu5LemA8hDjkgrf9o,lyre-lm,"[{'index': 0, 'turns': ['[Intro] Man, I'm so h...",1.700812e+09
3,3,YVAnF8UnSD8hUHPrXcqYjU,lyre-lm,"[{'index': 0, 'turns': ['[Verse 1: Liam] I'm s...",1.700811e+09
4,4,kJ8Pzc5ULRzLZRiYfsUXBy,lyre-lm,"[{'index': 0, 'turns': ['[Intro] Yeah, uh Uh Y...",1.700813e+09
...,...,...,...,...,...
676,676,HN47hTuTDsoRbRP3xhRpef,lyre-lm,"[{'index': 0, 'turns': ['[Intro] Oh, oh, oh, o...",1.700810e+09
677,677,jLgZA84z59fA68tbLTgrmW,lyre-lm,"[{'index': 0, 'turns': ['[Verse 1: Jesy] Baby,...",1.700806e+09
678,678,VhtLnhbNQ2iwmPMDoe8Wcj,lyre-lm,"[{'index': 0, 'turns': ['[Intro: Kirko Bangz] ...",1.700808e+09
679,679,MCxUTWaTAYkuMwF65uzA3a,lyre-lm,"[{'index': 0, 'turns': ['[Intro: Quavo] Yo, wh...",1.700815e+09


In [12]:
def extract_response(s):
    return s[0]['turns'][0]

In [13]:
generated['model_response'] = generated['choices'].map(extract_response)
generated

,question_id,answer_id,model_id,choices,tstamp,model_response
0,0,KxXAZ6LaGeLrTBAPTZxgu2,lyre-lm,"[{'index': 0, 'turns': ['[Intro: Offset & 21 ...",1.700809e+09,[Intro: Offset & \n21 Savage\n]\n(\nIf Young M...
1,1,LMAq8FLz2p4etTLHiJ9m8i,lyre-lm,"[{'index': 0, 'turns': ['[Intro] ( I made it )...",1.700809e+09,"[Intro]\n(\nI made it\n)\n[Chorus]\nI made it,..."
2,2,BP8pScu5LemA8hDjkgrf9o,lyre-lm,"[{'index': 0, 'turns': ['[Intro] Man, I'm so h...",1.700812e+09,"[Intro]\nMan, I'm so high right now\nYou can't..."
3,3,YVAnF8UnSD8hUHPrXcqYjU,lyre-lm,"[{'index': 0, 'turns': ['[Verse 1: Liam] I'm s...",1.700811e+09,[Verse 1: Liam]\nI'm sorry I'm not where you t...
4,4,kJ8Pzc5ULRzLZRiYfsUXBy,lyre-lm,"[{'index': 0, 'turns': ['[Intro] Yeah, uh Uh Y...",1.700813e+09,"[Intro]\nYeah, uh\nUh\nYou know what it is, yo..."
...,...,...,...,...,...,...
676,676,HN47hTuTDsoRbRP3xhRpef,lyre-lm,"[{'index': 0, 'turns': ['[Intro] Oh, oh, oh, o...",1.700810e+09,"[Intro]\nOh, oh, oh, oh\nOh, oh, oh, oh\nOh, o..."
677,677,jLgZA84z59fA68tbLTgrmW,lyre-lm,"[{'index': 0, 'turns': ['[Verse 1: Jesy] Baby,...",1.700806e+09,"[Verse 1: Jesy]\nBaby, you got me like i-i-i-i..."
678,678,VhtLnhbNQ2iwmPMDoe8Wcj,lyre-lm,"[{'index': 0, 'turns': ['[Intro: Kirko Bangz] ...",1.700808e+09,"[Intro: Kirko Bangz]\nSound M.O.B.\nYeah, yeah..."
679,679,MCxUTWaTAYkuMwF65uzA3a,lyre-lm,"[{'index': 0, 'turns': ['[Intro: Quavo] Yo, wh...",1.700815e+09,"[Intro: Quavo]\nYo, when we say ""Hannah Montan..."


In [16]:
together = lyric.merge(generated[['question_id', 'model_response']], how='inner', left_on='id', right_on='question_id')
together

,id,conversations,prompt,target_response,question_id,model_response
0,0,"[{'from': 'human', 'value': 'Write me a song c...",Write me a song called HETHEN with the followi...,[Intro: Offset]\nSo icy\nYeah\n[Chorus: Offset...,0,[Intro: Offset & \n21 Savage\n]\n(\nIf Young M...
1,1,"[{'from': 'human', 'value': 'Write me a song c...",Write me a song called Why Would I Stop? with ...,"[Intro]\nYeah, woah\nThis what they been waiti...",1,"[Intro]\n(\nI made it\n)\n[Chorus]\nI made it,..."
2,2,"[{'from': 'human', 'value': 'Write me a song c...",Write me a song called Still Got That Rock wit...,[Verse]\nNigga I be hard on a ho 'cause a bitc...,2,"[Intro]\nMan, I'm so high right now\nYou can't..."
3,3,"[{'from': 'human', 'value': 'Write me a song c...",Write me a song called Little Things with the ...,"[Verse 1: Zayn, \nZayn & Liam\n, \nLiam\n]\nYo...",3,[Verse 1: Liam]\nI'm sorry I'm not where you t...
4,4,"[{'from': 'human', 'value': 'Write me a song c...",Write me a song called Light Up (Cover) with t...,[Verse 1: Lil' Pope]\nOnly videos connect me ...,4,"[Intro]\nYeah, uh\nUh\nYou know what it is, yo..."
...,...,...,...,...,...,...
676,676,"[{'from': 'human', 'value': 'Write me a song c...",Write me a song called Trumpets with the follo...,[Chorus]\nEvery time that you get undressed\nI...,676,"[Intro]\nOh, oh, oh, oh\nOh, oh, oh, oh\nOh, o..."
677,677,"[{'from': 'human', 'value': 'Write me a song c...",Write me a song called Love on the Brain (Cove...,"[Verse: Jesy, \nAll\n & \nJade\n]\nAnd you got...",677,"[Verse 1: Jesy]\nBaby, you got me like i-i-i-i..."
678,678,"[{'from': 'human', 'value': 'Write me a song c...",Write me a song called In Her Lane with the fo...,"[Hook: Ty Dolla $ign]\nGirl, keep shitting on ...",678,"[Intro: Kirko Bangz]\nSound M.O.B.\nYeah, yeah..."
679,679,"[{'from': 'human', 'value': 'Write me a song c...",Write me a song called Fight Night with the fo...,"[Intro: Takeoff]\nIf you know me, know this ai...",679,"[Intro: Quavo]\nYo, when we say ""Hannah Montan..."


In [18]:
together[['id', 'prompt', 'model_response', 'target_response']].to_json("test_song_generations.json", orient='records', indent=2)